##### Taken from https://github.com/catherinedevlin/ipython-sql


# Examples of using SQL from within a python notebook


## 3 TABLE JOIN to get 'best employee'
```sql
SELECT Employees.FirstName, ROUND(SUM(OrderDetails.Quantity*Products.Price),2) AS CASH FROM OrderDetails
JOIN Orders
ON OrderDetails.OrderID = Orders.OrderID
JOIN Employees
ON Orders.EmployeeID = Employees.EmployeeID
JOIN Products
ON OrderDetails.ProductID = Products.ProductID
GROUP BY Employees.FirstName
ORDER BY CASH DESC;
```


## 5 TABLE JOIN to get tofu based income
```sql
SELECT orderdetails.quantity, products.productname, products.price, employees.firstname
FROM orderdetails
JOIN products
ON orderdetails.productID=products.productID
JOIN categories
ON products.categoryid=categories.categoryID
JOIN orders
ON orders.orderID=orderdetails.orderID
JOIN employees
ON employees.employeeID=orders.employeeID
WHERE productname LIKE '%Tofu%'
ORDER BY quantity DESC;
```

In [0]:
# ! runs a command in terminal (BASH)
!/ext/anaconda3/bin/pip install --user sql
!/ext/anaconda3/bin/pip install --user ipython-sql
# RUN ONLY ONCE

In [3]:
%load_ext sql

In [4]:
%sql sqlite://

'Connected: None@None'

In [5]:
%%sql
CREATE TABLE writer (first_name, last_name, year_of_death);

Done.


[]

In [6]:
%%sql
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);
INSERT INTO writer VALUES ('Berthold', 'Brecht', 1956);


1 rows affected.
1 rows affected.


[]

In [7]:
%sql select * from writer

Done.


first_name,last_name,year_of_death
William,Shakespeare,1616
Berthold,Brecht,1956


In [0]:
# EXAMPLE FROM LANTHE
%%sql
DROP TABLE IF EXISTS Mutations;
DROP TABLE IF EXISTS Samples;
CREATE TABLE Mutations (mutation_id INTEGER PRIMARY KEY, sample_id INT, chromosome VARCHAR(10), position VARCHAR(20), GT VARCHAR(20));
CREATE TABLE Samples (sample_id INTEGER PRIMARY KEY, description VARCHAR(255));

In [0]:
#EXAMPLE FROM FRANKA
vcf_reader = vcf.Reader(open("PIK3CA.vcf", 'r'))
test = 0
mutation_id = 0

# ONLY ADD 20 ENTRIES
while test < 20:
    for record in vcf_reader:
        
        # add mutation to Mutation table
        mutation_id += 1
        command_string = "INSERT INTO Mutations VALUES ({}, \'{}\', \'{}\', {}, \'{}\', \'{}\')".format(mutation_id, record.var_type, record.CHROM, record.POS, record.REF, record.ALT)
        %sql $command_string
        
        # add mutation-sample pairs to Mutation2Sample table
        for sample in record.samples:
            if (sample['GT'] != '0/0') and (sample['GT'] != './.'):
                command_string = "INSERT INTO Mutation2Sample VALUES ({}, \'{}\')".format(mutation_id, sample.sample)
                %sql $command_string
        
        test += 1